## Var declarations

In [16]:
data_file_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/data/merged_data_production_bug_reports.json"

## General methods

In [17]:
import json
import os

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def dict_to_json_file(file, dic):
    folder = os.path.dirname(file)
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(file), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

In [21]:
def get_file_name_from_stack_trace(stack_trace_entry):
    try: #  Treating exceptions such as Unknow Source and Native Method
        parentheses_opening_index = stack_trace_entry.index('(')
        colon_index = stack_trace_entry.index(':',parentheses_opening_index + 1)
        file_name = stack_trace_entry[parentheses_opening_index + 1 : colon_index]
    except:
        parentheses_opening_index = stack_trace_entry.index('(')
        parentheses_closing_index = stack_trace_entry.index(')',parentheses_opening_index + 1)
        file_name = stack_trace_entry[parentheses_opening_index + 1 : parentheses_closing_index]
    return file_name

def get_method_name_from_stack_trace(stack_trace_entry):
    parentheses_opening_index = stack_trace_entry.index('(')
    stack_trace_entry = stack_trace_entry[:parentheses_opening_index]
    last_dot_index = stack_trace_entry.rfind(".")
    method_name = stack_trace_entry[last_dot_index + 1 : ]
    return method_name

def get_file_line_from_stack_trace(stack_trace_entry):
    try: # Treating exceptions such as Unknow Source and Native Method
        colon_index = stack_trace_entry.index(':')
        parentheses_closing_index = stack_trace_entry.index(')', colon_index + 1)
        file_line = stack_trace_entry[colon_index + 1 : parentheses_closing_index]
        return file_line
    except:
        return -1

bugs_data = json_file_to_dict(data_file_path)
for project in bugs_data.keys():
    for bug_id in bugs_data[project].keys():
        stack_trace_entries = []
        for stack_trace in bugs_data[project][bug_id]["log"]:
            stack_trace_entries += stack_trace.replace("\t", "").split("\n")
        st_methods = []
        st_files = []
        st_lines = []
        for st_entry in stack_trace_entries:
            try:
                method_name = get_method_name_from_stack_trace(st_entry)
                file_name = get_file_name_from_stack_trace(st_entry)
                line_number = get_file_line_from_stack_trace(st_entry)
                st_methods.append(method_name)
                st_files.append(file_name)
                st_lines.append(line_number)
            except:
                continue
        bugs_data[project][bug_id]["stack_trace_methods"] = st_methods
        bugs_data[project][bug_id]["stack_trace_files"] = st_files
        bugs_data[project][bug_id]["stack_trace_lines"] = st_lines
dict_to_json_file(data_file_path,bugs_data)
print("Done")

Done
